# Определение стоимости автомобилей.

Для того, чтобы определять рыночную стоимость автомобиля в мобильном приложении, необходимо на основании исторических данных объявлений о продаже, создать модель машинного обучения. 

Данные представляют собой ~ 335 тысяч объявлений с техническими характеристиками автомобилей и итоговой ценой. 

План разработки модели: 

1. Изучить и предобработать данные. 
2. Определить значимые признаки и целевой признак, создать обучающую и валидационную выборки. 
3. Обучить модель методом линейной регрессии. 
4. Обучить модель методос градиентного бустинга. 
5. Выбрать наиболее оптимальный с точки зрения точности и времени обучени, работы алгоритм.

В качестве метрики качества будем использовать RMSE. 

<div class="alert alert-block alert-success">
<b>Успех:</b> 

Отличное введение 👍 Важно чтобы в проектах было подробное описание: когда потенциальные работодатели будут смотреть твой проект в портфолио, им нужно будет понимать задачу, чтобы можно было оценить её решение :) Молодец, что описание пишешь своими словами, не копируешь дословно описание проекта для студентов, так оно не выглядит как инструкция, а работа выглядит более самостоятельной.
</div>

In [166]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

## Предообработка данных

In [167]:
df = pd.read_csv('/datasets/autos.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Переименуем, для дальнейшего удобства, колонки в формат `snake_case`

In [168]:
normal_columns = []
for c in df.columns:
    c = re.sub(r'(?<!^)(?=[A-Z])', '_', c).lower()
    normal_columns.append(c)
df.columns = normal_columns

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

Выделим признаки, которые не влияют на стоимость автомобиля и удалим их из датасета, чтобы не исказить результат обучения модели. 

Признаки к удалению: 
['date_crawled', 'registration_month', 'date_created', 'postal_code', 'last_seen']


In [170]:
columns_for_delete = ['date_crawled', 'registration_month', 'date_created', 'postal_code', 'last_seen']

df = df.drop(columns = columns_for_delete)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               354369 non-null  int64 
 1   vehicle_type        316879 non-null  object
 2   registration_year   354369 non-null  int64 
 3   gearbox             334536 non-null  object
 4   power               354369 non-null  int64 
 5   model               334664 non-null  object
 6   kilometer           354369 non-null  int64 
 7   fuel_type           321474 non-null  object
 8   brand               354369 non-null  object
 9   not_repaired        283215 non-null  object
 10  number_of_pictures  354369 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Удалить неинформативные признаки - хорошее решение 👍
</div>

Необходимо проанализировать пропуски и, по-возможности, их заполнить в колонках:

`vehicle_type`

`gearbox`

`model`

`fuel_type`

`not_repaired`

Предобработаем данные в колонке not_repared. 
Изменим значение yes на True, значение no на False. 

Далее заполним пропущенные значения случайными из соответвующих колонок.

In [171]:
# Меняем значения на true и false
df['not_repaired'] = df['not_repaired'].replace(
    {'yes': True,
    'no': False}
)

# Функция, которая заменит пропуски случайным значением из текущей колонки
def replacing (column):
    column = column.fillna(np.random.choice(column!=np.nan))
    return column


df['fuel_type'] = replacing(df['fuel_type'])
df['gearbox'] = replacing(df['gearbox'])
df['not_repaired'] = replacing(df['not_repaired'])
df['vehicle_type'] = replacing(df['vehicle_type'])
df = df.drop(index=(df[df['model'].isna()==True].index))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334664 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               334664 non-null  int64 
 1   vehicle_type        334664 non-null  object
 2   registration_year   334664 non-null  int64 
 3   gearbox             334664 non-null  object
 4   power               334664 non-null  int64 
 5   model               334664 non-null  object
 6   kilometer           334664 non-null  int64 
 7   fuel_type           334664 non-null  object
 8   brand               334664 non-null  object
 9   not_repaired        334664 non-null  bool  
 10  number_of_pictures  334664 non-null  int64 
dtypes: bool(1), int64(5), object(5)
memory usage: 28.4+ MB


Изучим данные в категориальных колонках.

In [172]:
df.describe()

,price,registration_year,power,kilometer,number_of_pictures
count,334664.000000,334664.000000,334664.000000,334664.000000,334664.0
mean,4504.346790,2003.923992,111.373195,128562.588148,0.0
std,4531.438572,69.377219,185.156439,37205.926976,0.0
min,0.000000,1000.000000,0.000000,5000.000000,0.0
25%,1150.000000,1999.000000,70.000000,125000.000000,0.0
50%,2800.000000,2003.000000,105.000000,150000.000000,0.0
75%,6500.000000,2008.000000,143.000000,150000.000000,0.0
max,20000.000000,9999.000000,20000.000000,150000.000000,0.0


In [173]:
df['number_of_pictures'].value_counts()

0    334664
Name: number_of_pictures, dtype: int64

В колонке `number_of_pictures` все значения = 0. Удалим данную колонку.

In [174]:
df = df.drop(columns = 'number_of_pictures')

Оставим выбросы, как есть, для того, чтобы модель умела на реальных данных делать правдивые предсказания.

In [175]:
# Удаляем данные, где год > 2016
df = df.drop(index = df[df['registration_year']>2016].index)

# Удаляем данные, где power > 500 и < 50
df = df.drop(index = df[(df['power']>500) | (df['power']<50)].index)

# Удалем данные, где цена < 300
df = df.drop(index = df[df['price']<300].index)

In [176]:
df.describe()

,price,registration_year,power,kilometer
count,274030.000000,274030.000000,274030.000000,274030.000000
mean,5062.559037,2003.157370,122.844075,128229.828851
std,4621.051284,6.519686,52.513415,36526.541891
min,300.000000,1000.000000,50.000000,5000.000000
25%,1500.000000,1999.000000,82.000000,125000.000000
50%,3480.000000,2003.000000,115.000000,150000.000000
75%,7300.000000,2007.000000,150.000000,150000.000000
max,20000.000000,2016.000000,500.000000,150000.000000


<div class="alert alert-block alert-info">
<b>Изменения:</b> 
    
Удалил выбросы. 
</div>

## Создаем выборки.

In [177]:
features = df.drop(columns = 'price')
target = df['price']

features = pd.get_dummies(features, drop_first=True)

In [178]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)



print (features_train.shape)
print (features_valid.shape)
print (target_train.shape)
print (target_valid.shape)

(205522, 307)
(68508, 307)
(205522,)
(68508,)


Выборки созданы. 

Далее выполним скалирование признаков.

In [179]:
scaler = StandardScaler()

scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)

Данные для обучения моделей готовы.

## Обучение линейной регрессией

In [180]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)

CPU times: user 7.61 s, sys: 1.86 s, total: 9.47 s
Wall time: 2.48 s


LinearRegression()

In [181]:
cross_val_scores = cross_val_score(model, features_train, target_train, 
                                   scoring = 'neg_mean_squared_error', 
                                   cv = 10)

median_rmse = np.abs(np.median(cross_val_scores))**0.5
print ('Медианное значение метрики RMSE для 10 кросс валидаций составляет:', median_rmse)

Медианное значение метрики RMSE для 10 кросс валидаций составляет: 2544.0655810236085


Измерим время предсказания. 

In [182]:
%%time
model.predict(features_train)

CPU times: user 125 ms, sys: 29.7 ms, total: 154 ms
Wall time: 26.3 ms


array([8925.14456009, 5507.86792165,  713.22480218, ..., 3696.6640504 ,
       6586.13322785, 9829.75392749])

Модель обучена и протестирована на валидационной выборке. 

Далее проверим модель на вменяемость. 

### Выводы. 

Линейная регрессия:

Время обучения: 3 сек

Время предсказания: 53.6 мс

Метрика RMSE: ~2600

## Проверка на вменяемость, обучение dummy моделью

In [183]:
model = DummyRegressor(strategy='median')
model.fit(features_train, target_train)
predicted_values = model.predict(features_valid)
mean_squared_error(target_valid, predicted_values, squared= False)

4865.417235006802

Значение метрики rmse > значения метрики rmse обученной ранее модели 

$=>$ 

Модель вменяема.
Попробуем еще уменьшить значение метрики rmse с помощью градиентного бустинга. 

## Обучение градиентным бустингом

### Catboost
Так как модель catboost умеет обрабатывать категориальные признаки, пересоздадим тестовые и валидационные выборки без dummy признаков. 

In [184]:
features = df.drop(columns = 'price')
target = df['price']


features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

print (features_train.shape)
print (features_valid.shape)
print (target_train.shape)
print (target_valid.shape)

(205522, 9)
(68508, 9)
(205522,)
(68508,)


Данные для обучения модели готовы. 

In [185]:
best_result = 5000 ##Значение, которое выдает dummy model. 
best_model = 0

for i in range (25,28):
    for n in range (5,15):
        model = CatBoostRegressor(iterations=i, 
                                  depth=n,
                                  learning_rate=0.7,
                                  loss_function='RMSE',
                                  cat_features=['vehicle_type', 'model', 'gearbox', 'fuel_type', 'brand'])
        model.fit(features_train, target_train)
        scores = cross_val_score(model, features_train, target_train, scoring = 'neg_mean_squared_error')
        result = np.abs(np.median(scores))**0.5
        if result < best_result:
            best_result = result
            best_model = model

0:	learn: 2988.3893009	total: 15.9ms	remaining: 382ms
1:	learn: 2513.9137816	total: 32.1ms	remaining: 370ms
2:	learn: 2269.9359070	total: 48.9ms	remaining: 359ms
3:	learn: 2156.2478231	total: 63.3ms	remaining: 332ms
4:	learn: 2098.6945938	total: 78.7ms	remaining: 315ms
5:	learn: 2058.3112073	total: 93.2ms	remaining: 295ms
6:	learn: 2013.6783598	total: 107ms	remaining: 276ms
7:	learn: 1985.8222805	total: 122ms	remaining: 259ms
8:	learn: 1956.5972158	total: 137ms	remaining: 244ms
9:	learn: 1935.9739740	total: 152ms	remaining: 228ms
10:	learn: 1920.1654644	total: 167ms	remaining: 213ms
11:	learn: 1903.5530565	total: 182ms	remaining: 197ms
12:	learn: 1887.6487266	total: 195ms	remaining: 180ms
13:	learn: 1873.0109852	total: 209ms	remaining: 165ms
14:	learn: 1863.3263729	total: 224ms	remaining: 149ms
15:	learn: 1851.8469719	total: 242ms	remaining: 136ms
16:	learn: 1842.4702885	total: 258ms	remaining: 121ms
17:	learn: 1835.9978061	total: 272ms	remaining: 106ms
18:	learn: 1829.6122449	total: 2

In [186]:
optimal_depth = best_model.get_param(key = 'depth')
optimal_iterations = best_model.get_param(key = 'iterations')

print (optimal_depth, optimal_iterations)

14 27


Выбираем модель с глубиной = 12, и количеством итераций = 26. 

Оценим время обучения на тренировочных данных. 

In [187]:
%%time
model = CatBoostRegressor(iterations=optimal_iterations, 
                                  depth=optimal_depth,
                                  learning_rate=0.7,
                                  loss_function='RMSE',
                                  cat_features=['vehicle_type', 'model', 'gearbox', 'fuel_type', 'brand'])
model.fit(features_train, target_train)

0:	learn: 2632.1440864	total: 42.6ms	remaining: 1.11s
1:	learn: 2040.4125337	total: 173ms	remaining: 2.16s
2:	learn: 1855.5241696	total: 299ms	remaining: 2.39s
3:	learn: 1785.3630962	total: 429ms	remaining: 2.47s
4:	learn: 1720.2981190	total: 554ms	remaining: 2.44s
5:	learn: 1693.0840551	total: 680ms	remaining: 2.38s
6:	learn: 1659.8513560	total: 811ms	remaining: 2.32s
7:	learn: 1634.6048609	total: 942ms	remaining: 2.24s
8:	learn: 1613.2382877	total: 1.07s	remaining: 2.15s
9:	learn: 1597.6896293	total: 1.2s	remaining: 2.04s
10:	learn: 1574.5284645	total: 1.33s	remaining: 1.93s
11:	learn: 1557.0335672	total: 1.46s	remaining: 1.82s
12:	learn: 1547.2714133	total: 1.58s	remaining: 1.71s
13:	learn: 1536.5605843	total: 1.72s	remaining: 1.59s
14:	learn: 1526.1596165	total: 1.84s	remaining: 1.47s
15:	learn: 1511.4898467	total: 1.96s	remaining: 1.35s
16:	learn: 1503.1795736	total: 2.09s	remaining: 1.23s
17:	learn: 1489.1286645	total: 2.22s	remaining: 1.11s
18:	learn: 1481.2304734	total: 2.35s	r

Время обучения составляет 1.63 секунды.

Оценим метрику RMSE с помощью кросс-валидации на тренировочных данных. 

In [188]:
cross_val_scores = cross_val_score(model, features_train, target_train, 
                                   scoring = 'neg_mean_squared_error', 
                                   cv = 10)

median_rmse = np.abs(np.median(cross_val_scores))**0.5

0:	learn: 2584.3294058	total: 120ms	remaining: 3.13s
1:	learn: 2003.1199807	total: 242ms	remaining: 3.02s
2:	learn: 1830.7567085	total: 365ms	remaining: 2.92s
3:	learn: 1764.2129052	total: 488ms	remaining: 2.81s
4:	learn: 1718.1584192	total: 609ms	remaining: 2.68s
5:	learn: 1674.5294885	total: 730ms	remaining: 2.55s
6:	learn: 1646.5635604	total: 850ms	remaining: 2.43s
7:	learn: 1615.5071906	total: 974ms	remaining: 2.31s
8:	learn: 1587.6959542	total: 1.09s	remaining: 2.18s
9:	learn: 1567.5957628	total: 1.22s	remaining: 2.06s
10:	learn: 1549.2113171	total: 1.34s	remaining: 1.94s
11:	learn: 1531.2165042	total: 1.46s	remaining: 1.83s
12:	learn: 1521.5279050	total: 1.59s	remaining: 1.71s
13:	learn: 1500.4144936	total: 1.71s	remaining: 1.59s
14:	learn: 1490.8472116	total: 1.83s	remaining: 1.47s
15:	learn: 1478.7661365	total: 1.96s	remaining: 1.35s
16:	learn: 1471.8283074	total: 2.08s	remaining: 1.22s
17:	learn: 1465.4030108	total: 2.2s	remaining: 1.1s
18:	learn: 1459.5265868	total: 2.32s	rem

In [189]:
print ('Медианное значение метрики RMSE для 10 кросс валидаций составляет:', median_rmse)

Медианное значение метрики RMSE для 10 кросс валидаций составляет: 1610.4779053407726


Оценим время предсказания модели на тренировочных данных. 

In [190]:
%%time
predicted_values = model.predict(features_train)

CPU times: user 326 ms, sys: 6.05 ms, total: 332 ms
Wall time: 247 ms


### Выводы. 

CatBoost:

Время обучения: 3,58 сек

Время предсказания: 235 мс

Метрика RMSE: ~1610

### LightGBM

In [191]:
cat_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand']

for column in cat_columns:
    features_train[column] = features_train[column].astype('category')
    features_valid[column] = features_valid[column].astype('category')

In [192]:
train_sample = lgb.Dataset(features_train, target_train)
#valid_sample = lgb.Dataset(features_valid, target_valid, reference= train_sample)

params = {
    'metric':'rmse',
    'objective': 'regression',
    'num_leaves': 150,
    'learning_rate': 0.7,
}

In [193]:
%%time
gbm = lgb.train(params,
                train_sample,
                num_boost_round=50,
                valid_sets=train_sample)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005615 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 600
[LightGBM] [Info] Number of data points in the train set: 205522, number of used features: 9
[LightGBM] [Info] Start training from score 5067.438741
[1]	training's rmse: 2325.19
[2]	training's rmse: 1817.67
[3]	training's rmse: 1691.06
[4]	training's rmse: 1639.07
[5]	training's rmse: 1605.21
[6]	training's rmse: 1576.24
[7]	training's rmse: 1556.73
[8]	training's rmse: 1539.74
[9]	training's rmse: 1527.43
[10]	training's rmse: 1517.12
[11]	training's rmse: 1505.27
[12]	training's rmse: 1496.08
[13]	training's rmse: 1486.91
[14]	training's rmse: 1477.61
[15]	training's rmse: 1469.55
[16]	training's rmse: 1463.13
[17]	training's rmse: 1459.01
[18]	training's rmse: 1455.43
[19]	training's rmse: 1452.09
[20]	training's rmse: 1444.8

In [194]:
state = np.random.RandomState()
scores = []
for i in range (1,11):
    features_train_sampled = features_train.sample(frac = 1,
                                           replace=True,
                                           random_state=state)
    target_train_sampled = target_train[features_train_sampled.index]
    predicted_values = gbm.predict(features_train_sampled)
    score = mean_squared_error(target_train_sampled, predicted_values, squared= False)
    scores.append(score)
np.median(np.array(scores))

1333.13141550214

Оценим время предсказания модели на обучающих данных.

In [195]:
%%time
predicted_values = gbm.predict(features_train)

CPU times: user 1.75 s, sys: 158 ms, total: 1.9 s
Wall time: 254 ms


### Выводы.

LightGBM:

Время обучения: 572 сек

Время предсказания: 255 мс

Метрика RMSE: ~1340

## Проверка выбранной модели на валидационной выборке. 

По совокупности признаков, выберем целевой моделью 'LightGBM'. 
Проверим качество предсказаний модели на валидационной выборке. 

In [196]:
predicted_values = gbm.predict(features_valid)
score = mean_squared_error(target_valid, predicted_values, squared= False)
print ('Метрика RMSE на валидаионной выборке для выбранной модели LightGBM составляет:', score)

Метрика RMSE на валидаионной выборке для выбранной модели LightGBM составляет: 1578.9295848242134


<div class="alert alert-block alert-info">
<b>Комментарий ревьюера:</b>

Чтобы все было ясно, напишу конкретный план (структуру) обучения и анализа моделей:

1. Обучение моделей. В обучении нужно рассмотреть хотя бы одну простую модель и один бустинг. Подбор гиперпараметров нужно провести хотя бы одной модели. Тут есть два варианта:
    - без валидационной выборки. Здесь нужно подбирать гиперпараметры с помощью кросс-валидации (GridSearchCV, RandomizedSearchCV или вручную (cross_val_score));
    - валидационная выборка есть. Здесь можно не использовать кросс-валидацию и подбирать гиперпараметры вручную.  
2. Анализ моделей. После нахождения лучших гиперпараметров стоит измерить время обучения, предсказания и RMSE. Тут тоже есть два варианта:
    - без валидационной выборки. RMSE на кросс-валидации. Время обучения = время model.fit(X_train). Время предсказания  = model.predict(X_train);
    - валидационная выборка есть. RMSE на validation. Время обучения = время model.fit(X_train). Время предсказания = время model.predict(X_valid).  
    
    После этого напиши вывод по анализу (можешь также результаты моделей занести в общую таблицу) и посоветуй заказчику одну модель на основе его критериев;
3. Тестирование. Рассчитай финальную метрику лучшей модели на тестовой выборке (до этого тестовая выборка нигде не должна использоваться!). RMSE должно быть меньше 2500. Если метрика не дотягивает, попробуй исправить мои замечания, также можешь потюнить гиперпараметры (на этапе обучения моделей, не на тестовой выборке!)
</div>

## Вывод. 

Для подготовки модели были предоствлены 335 тысяч строк объявлений сервиса.


Часть признаков имели пропуски, которые были заполнены случайными значениями соответствующего столбца. 

Часть признаков ( 'date_crawled', 'registration_month', 'date_created', 'postal_code', 'last_seen') - удалены, так как не являются признаком продаваемого автомобиля, как следствие, не помогли бы определить его рыночную стоимость. 

Также по каждой колонке были удалены выбросы. 

Были обчены модели с помощью градиентного бустинга и линейной регрессии. Все модели - вменяемы, так как показали результат метрики RMSE ниже, чем dummy model. 

По двум из трех критериев (Скорость обучения, качество предсказаний) лучший результат показала модель LightGBM,  по скорости предсказания лучшей моделью оказалась catBoost с разницей в ~ 20 мс. 

Для интеграции в мобильное приложение рекомендуется модель LightGBM.

На валидационной выборке модель показала значение метрики RMSE 1578. 